## Kiwi 토크나이저를 이용한 BM25 리트리버

In [2]:
# 라이브러리 업데이트
!pip install -qU langchain-teddynote


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
# 비교를 위한 BM25Retriever
from langchain_community.retrievers import BM25Retriever

# 커스텀 구현한 한국어 형태소 분석기(Kiwi)를 사용한 BM25Retriever
from langchain_teddynote.retrievers import KiwiBM25Retriever

sample_texts = [
    "금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.",
    "금융저축산물보험은 장기적인 저축 목적과 더불어, 축산물 제공 기능을 갖추고 있는 특별 금융 상품입니다.",
    "금융보씨 험한말 좀 하지마시고, 저축이나 좀 하시던가요. 뭐가 그리 급하신지 모르겠네요.",
    "금융단폭격보험은 저축은 커녕 위험 대비에 초점을 맞춘 상품입니다. 높은 위험을 감수하고자 하는 고객에게 적합합니다.",
]

In [4]:
# 리트리버를 생성합니다.
kiwi = KiwiBM25Retriever.from_texts(sample_texts)

In [5]:
# 유사도 검색을 수행합니다.
kiwi.invoke("금융보험")

[Document(page_content='금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.'),
 Document(page_content='금융저축산물보험은 장기적인 저축 목적과 더불어, 축산물 제공 기능을 갖추고 있는 특별 금융 상품입니다.'),
 Document(page_content='금융단폭격보험은 저축은 커녕 위험 대비에 초점을 맞춘 상품입니다. 높은 위험을 감수하고자 하는 고객에게 적합합니다.'),
 Document(page_content='금융보씨 험한말 좀 하지마시고, 저축이나 좀 하시던가요. 뭐가 그리 급하신지 모르겠네요.')]

개인적인 필요에 의하여 유사도 점수를 계산하고 metadata 에 `score` 를 추가하는 기능을 추가하였습니다.

In [6]:
# 유사도 검색을 수행합니다.
kiwi.search_with_score("금융보험")

(Document(metadata={'score': 0.2672729220215512}, page_content='금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.'),
 Document(metadata={'score': 0.2615968670569818}, page_content='금융저축산물보험은 장기적인 저축 목적과 더불어, 축산물 제공 기능을 갖추고 있는 특별 금융 상품입니다.'),
 Document(metadata={'score': 0.24633295918203968}, page_content='금융단폭격보험은 저축은 커녕 위험 대비에 초점을 맞춘 상품입니다. 높은 위험을 감수하고자 하는 고객에게 적합합니다.'),
 Document(metadata={'score': 0.22479725173942738}, page_content='금융보씨 험한말 좀 하지마시고, 저축이나 좀 하시던가요. 뭐가 그리 급하신지 모르겠네요.'))

In [7]:
bm25 = BM25Retriever.from_texts(sample_texts)

# BM25 KiwiBM25 비교
print(f'Kiwi:  \t {kiwi.invoke("금융보험")[0].page_content}')
print(f'BM25:  \t {bm25.invoke("금융보험")[0].page_content}')

Kiwi:  	 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
BM25:  	 금융단폭격보험은 저축은 커녕 위험 대비에 초점을 맞춘 상품입니다. 높은 위험을 감수하고자 하는 고객에게 적합합니다.


k 값 설정

In [8]:
# 리트리버를 생성합니다.
kiwi = KiwiBM25Retriever.from_texts(sample_texts)
kiwi.k = 2
# 유사도 검색을 수행합니다.
kiwi.search_with_score("금융보험")

(Document(metadata={'score': 0.2672729220215512}, page_content='금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.'),
 Document(metadata={'score': 0.2615968670569818}, page_content='금융저축산물보험은 장기적인 저축 목적과 더불어, 축산물 제공 기능을 갖추고 있는 특별 금융 상품입니다.'))

## Konlpy(Kkma, Okt) 사용한 BM25Retriever

In [9]:
# 커스텀 구현한 한국어 형태소 분석기를 사용한 BM25Retriever
from langchain_teddynote.retrievers import (
    KkmaBM25Retriever,
    OktBM25Retriever,
)

In [10]:
# 리트리버를 생성합니다.
kkma = KkmaBM25Retriever.from_texts(sample_texts)
okt = OktBM25Retriever.from_texts(sample_texts)

In [11]:
# kkma + BM25
kkma.invoke("금융보험")

[Document(page_content='금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.'),
 Document(page_content='금융저축산물보험은 장기적인 저축 목적과 더불어, 축산물 제공 기능을 갖추고 있는 특별 금융 상품입니다.'),
 Document(page_content='금융보씨 험한말 좀 하지마시고, 저축이나 좀 하시던가요. 뭐가 그리 급하신지 모르겠네요.'),
 Document(page_content='금융단폭격보험은 저축은 커녕 위험 대비에 초점을 맞춘 상품입니다. 높은 위험을 감수하고자 하는 고객에게 적합합니다.')]

In [12]:
# 유사도 검색을 수행합니다.
kkma.search_with_score("금융보험")

(Document(metadata={'score': 0.2770318309036624}, page_content='금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.'),
 Document(metadata={'score': 0.27008606866154405}, page_content='금융저축산물보험은 장기적인 저축 목적과 더불어, 축산물 제공 기능을 갖추고 있는 특별 금융 상품입니다.'),
 Document(metadata={'score': 0.22683710948352356}, page_content='금융보씨 험한말 좀 하지마시고, 저축이나 좀 하시던가요. 뭐가 그리 급하신지 모르겠네요.'),
 Document(metadata={'score': 0.22604499095126998}, page_content='금융단폭격보험은 저축은 커녕 위험 대비에 초점을 맞춘 상품입니다. 높은 위험을 감수하고자 하는 고객에게 적합합니다.'))

In [13]:
# okt + BM25
okt.invoke("금융보험")

[Document(page_content='금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.'),
 Document(page_content='금융저축산물보험은 장기적인 저축 목적과 더불어, 축산물 제공 기능을 갖추고 있는 특별 금융 상품입니다.'),
 Document(page_content='금융단폭격보험은 저축은 커녕 위험 대비에 초점을 맞춘 상품입니다. 높은 위험을 감수하고자 하는 고객에게 적합합니다.'),
 Document(page_content='금융보씨 험한말 좀 하지마시고, 저축이나 좀 하시던가요. 뭐가 그리 급하신지 모르겠네요.')]

In [14]:
# 유사도 검색을 수행합니다.
okt.search_with_score("금융보험")

[Document(metadata={'score': 0.2671682763745754}, page_content='금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.'),
 Document(metadata={'score': 0.26201826091225167}, page_content='금융저축산물보험은 장기적인 저축 목적과 더불어, 축산물 제공 기능을 갖추고 있는 특별 금융 상품입니다.'),
 Document(metadata={'score': 0.24596984920046247}, page_content='금융단폭격보험은 저축은 커녕 위험 대비에 초점을 맞춘 상품입니다. 높은 위험을 감수하고자 하는 고객에게 적합합니다.'),
 Document(metadata={'score': 0.22484361351271037}, page_content='금융보씨 험한말 좀 하지마시고, 저축이나 좀 하시던가요. 뭐가 그리 급하신지 모르겠네요.')]